<a href="https://www.kaggle.com/code/pmykola/lending-club-loan-default-prediction?scriptVersionId=103801415" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Modeling Credit Risk with Lending Club data

Internal name: KP102


### Outline

1. Business problem and summary of results
2. Data import.
3. Data preprocessing.
4. EDA.
5. Sample split, missing values and feature engineering.
6. Modeling.
7. Interpretation of the results.
8. Business implications.



### 1. Business problem

Lending Club was a peer to peer lending platform. Over 2007-2020 it issued at least 2.9M loans. Loans varied in size between \\$1,000 and \\$40,000 and were used to refinance consumer loans or to cover midsize expenses.

After accepting an application for a loan, Lending Club used internal risk model to assign credit rating and an interest rate to a loan. Then it placed the loan on online platform, where investors could invest in it. Investors fully bore credit risk in return for getting interest payments. 

Thus the key problem of investors in Lending Club was to determine riskiness of a loan. Investors' objective is to avoid risky loans, in which interest rate is not enough to cover expected credit loss. While Lending Club no longer engages in peer to peer lending, there is a number of P2P lending platforms, operating right now. The goal of this project is to solve investors' problem by building ML model.


#### Objective: Predict delinquency of a loan borrower.


#### Metric: Precision at 10% Recall.


#### Summary of Results
​
**I build XGBoost model to predict loan delinquencies. The model has 60.3% precision at 10% recall. It allows Lending Club investors to save \\$59.7M by avoiding the riskiest loans. The savings come from allocating investment funds into risk-free rate instead of the riskiest loans.** 


##### Notes:

One big question about this data is which features are available at the origination time and are never updated afterwards. It seems that most features are pulled at loan application/origination. 
The only features, pulled later, belong to the two groups:
- features, related to loan performance/payments.
- features, which clearly mention this in name/description. E.g., word 'last' in feature name.

I will use only features, known before loan issuance.

In [ ]:
!pip install pycodestyle

## 2. Data import

To make analysis manageable, I use 30% sample of the original dataset with 2.9M loans. This is a random sample. If you ramdomly sample original data and use this code, you should get very similar results.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import os, time, warnings, gzip, gc, random, math, shap, pickle, optuna, csv, sys, re
from IPython.display import display
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, KFold
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, precision_recall_curve, auc
from sklearn.metrics import mean_squared_error, mean_absolute_error, roc_auc_score
from sklearn.inspection import permutation_importance
from category_encoders import MEstimateEncoder
from xgboost import XGBClassifier

pd.set_option('display.max_columns', 5000)
pd.set_option('display.max_rows', 400)

warnings.filterwarnings("ignore")

In [ ]:
### target encoding ###
# source: https://www.kaggle.com/code/ryanholbrook/feature-engineering-for-house-prices/notebook

class CrossFoldEncoder:
    def __init__(self, encoder, **kwargs):
        self.encoder_ = encoder
        self.kwargs_ = kwargs  # keyword arguments for the encoder
        self.cv_ = KFold(n_splits=4)

    # Fit an encoder on one split and transform the feature on the
    # other. Iterating over the splits in all folds gives a complete
    # transformation. We also now have one trained encoder on each
    # fold.
    def fit_transform(self, X, y, cols):
        self.fitted_encoders_ = []
        self.cols_ = cols
        X_encoded = []
        for idx_encode, idx_train in self.cv_.split(X):
            fitted_encoder = self.encoder_(cols=cols, **self.kwargs_)
            fitted_encoder.fit(
                X.iloc[idx_encode, :], y.iloc[idx_encode],
            )
            X_encoded.append(fitted_encoder.transform(X.iloc[idx_train, :])[cols])
            self.fitted_encoders_.append(fitted_encoder)
        X_encoded = pd.concat(X_encoded)
        X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
        return X_encoded

    # To transform the test data, average the encodings learned from
    # each fold.
    def transform(self, X):
        from functools import reduce

        X_encoded_list = []
        for fitted_encoder in self.fitted_encoders_:
            X_encoded = fitted_encoder.transform(X)
            X_encoded_list.append(X_encoded[self.cols_])
        X_encoded = reduce(
            lambda x, y: x.add(y, fill_value=0), X_encoded_list
        ) / len(X_encoded_list)
        X_encoded.columns = [name + "_encoded" for name in X_encoded.columns]
        return X_encoded

In [ ]:
time0 = time.time()

with open('../input/lc-800k-sample/LCLoans_141_800k.pkl', 'rb') as pickled_one:
    df = pickle.load(pickled_one)
    
display(df.head())

features_tokeep = ['id', 'loan_status',
 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment','issue_d',
 'purpose', 'title', 'initial_list_status', 'application_type',
 'grade', 'sub_grade', 'fico_range_high',
 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'zip_code', 'addr_state',
 'dti',           
 'verification_status', 
 'mo_sin_rcnt_tl', 'mths_since_last_delinq', 'mths_since_last_major_derog', 'mths_since_last_record',
 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq',
 'num_tl_op_past_12m', 
 'earliest_cr_line', 'inq_last_6mths', 'inq_fi', 'inq_last_12m',
 'open_acc', 'acc_open_past_24mths', 'mort_acc', 'total_acc',
 'avg_cur_bal', 'il_util', 'tot_cur_bal', 
 'revol_bal', 'revol_util', 'max_bal_bc', 'bc_open_to_buy', 'mo_sin_rcnt_rev_tl_op', 'num_actv_rev_tl', 'num_op_rev_tl', 'total_rev_hi_lim',               
 'delinq_2yrs', 'acc_now_delinq', 'delinq_amnt', 'pub_rec', 'pub_rec_bankruptcies',
 'annual_inc_joint', 'dti_joint', 'verification_status_joint',
 'total_bal_ex_mort', 'tot_coll_amt', 'tax_liens', 'percent_bc_gt_75', 'pct_tl_nvr_dlq', 
 'open_rv_12m', 'open_il_12m', 'num_tl_90g_dpd_24m', 'num_tl_30dpd', 'num_tl_120dpd_2m',
 'num_accts_ever_120_pd',
 'recoveries', 'total_rec_prncp', 'total_rec_int']

df = df[features_tokeep]
gc.collect()

recoveries = df[df.loan_status.isin(['Charged Off', 'Default'])][[
    'id', 'loan_status', 'recoveries', 'loan_amnt', 'int_rate', 'total_rec_prncp', 'total_rec_int']]

df.drop(columns = ['recoveries', 'total_rec_prncp', 'total_rec_int'], inplace=True)
# this removes all features, not known to investors ex ante.

df.drop(columns = ['il_util', 'max_bal_bc'], inplace=True)
# these are useful features, which I will preprocess later

df.issue_d = df.issue_d.astype('O')
df.issue_d = pd.to_datetime(df.issue_d, format='%b-%Y')
df['year_issued']=df.issue_d.dt.year
    
#df = df.sample(200000, random_state=1)
df.reset_index(inplace=True, drop=True)
display(df.shape, time.time()-time0, df.head())

## 3. Data preprocessing

I preprocess main features and create a target. To get better separation between repaid and defaulted loans I look only at loans, which are Fully paid, Charged off or in Default.

In [ ]:
### feature description:

#f_desc = pd.read_excel('../input/lending-club-20072020q1/LCDataDictionary.xlsx')
#f_desc.columns = ['colname','desc']
#display(f_desc.head())
#display(f_desc)
#display(f_desc.loc[f_desc.colname=='open_acc','desc'], f_desc.loc[f_desc.colname=='total_acc','desc'])

In [ ]:
# remove some very rare loan types:

df = df[~df.purpose.isin(['educational', 'renewable_energy', 'wedding'])]
df.purpose = df.purpose.cat.remove_categories(['educational', 'renewable_energy', 'wedding'])


In [ ]:
# clean time features

df.earliest_cr_line = df.earliest_cr_line.astype('O')
df.earliest_cr_line = pd.to_datetime(df.earliest_cr_line, format='%b-%Y')
df['month_issued']=df.issue_d.dt.month
df['year_earliest']=df.issue_d.dt.year
df['years_borrowing'] = (df.issue_d - df.earliest_cr_line)/ np.timedelta64(1, 'Y')
df['pub_rec_pa'] = df.pub_rec/df.years_borrowing
display(df.head())

In [ ]:
# create a target variable

display(df.loan_status.value_counts())
df.target=np.nan
#df.loc[df.loan_status.isin(['Fully Paid', 'Does not meet the credit policy. Status:Fully Paid']), 'target']=0
#df.loc[df.loan_status.isin(['Charged Off', 'Late (31-120 days)', 'Does not meet the credit policy. Status:Charged Off', 'Default']), 'target']=1
df.loc[df.loan_status.isin(['Fully Paid']), 'target']=0
df.loc[df.loan_status.isin(['Charged Off', 'Default']), 'target']=1
df=df[df['target'].isin([0,1])]
display(df.shape,df.loan_status.value_counts(), df.count(), sys.getsizeof(df)/1048576)
df.drop(columns='loan_status',inplace=True)

#### Feature Preprocessing

There are many features, related to the loan size, which are measured in dollars. To make these features comparable across loans, we should rescale them by income of borrowers.

In [ ]:
# add key loan features, scaled by borrower's income:

df.loc[df.annual_inc<1,'annual_inc']=1
df['lti']=df.loan_amnt/df.annual_inc
df['iti']=(df.installment*12)/df.annual_inc
df.loc[df.lti==np.inf, 'lti']=np.nan
df.loc[df.lti>1.5, 'lti']=1.5
df.loc[df.iti==np.inf, 'iti']=np.nan
df.loc[df.iti>1, 'iti']=1
df.loc[df.revol_util>100,'revol_util']=100
df.loc[df.dti>100, 'dti']=100
df.loc[df.dti<0, 'dti']=0

df['revol_balance_income'] = df.revol_bal/df.annual_inc
df['avg_cur_bal_inc'] = df.avg_cur_bal/df.annual_inc
df['tot_cur_bal_inc'] = df.tot_cur_bal/df.annual_inc
df['total_bal_ex_mort_inc'] = df.total_bal_ex_mort/df.annual_inc
df['total_rev_inc'] = df.total_rev_hi_lim/df.annual_inc
df['open_cl_ratio']=df.open_acc/df.total_acc

# add more features

df['zip_code'] = df.zip_code.str.rstrip('xx').astype(int)
df['joint'] = df.dti_joint.notnull().astype(int)
df['emp_length'] = df.emp_length.str.rstrip(' years')
df.loc[df.emp_length=='< 1','emp_length'] = 0
df.loc[df.emp_length=='10+','emp_length'] = 10
df['emp_length'] = df.emp_length.astype(np.float32)
display(df.emp_length.value_counts())
df.amnt_same = (df.loan_amnt == df.funded_amnt_inv).astype(int)
df['low_fico'] = (df.fico_range_high<=659).astype(int)
df.loc[df.home_ownership.isin(['ANY','NONE','OTHER']), 'home_ownership'] = 'OTHER'
df['was_bankrupt'] = (df.pub_rec_bankruptcies>0).astype(int)

df.drop(columns = ['annual_inc_joint', 'dti_joint', 'verification_status_joint', 'earliest_cr_line', 'issue_d'], inplace=True)

df.head()

In [ ]:
# For features like 'time_since some credit event', treat NA as never and fill those values with 100 years equivalent

df.mo_sin_rcnt_tl = df.mo_sin_rcnt_tl.fillna(value=120)
df.num_tl_op_past_12m = df.num_tl_op_past_12m.fillna(value=0)

months_since_col = ['mths_since_last_delinq', 'mths_since_last_major_derog', 
                    'mths_since_last_record', 'mths_since_recent_bc_dlq', 'mths_since_recent_revol_delinq']

for col in months_since_col:
    df[col] = df[col].fillna(value=1200)

#display(df.count())

df.inq_fi = df.inq_fi.fillna(value=0)
df.inq_last_12m = df.inq_last_12m.fillna(value=0)

In [ ]:
print('Bankruptcy incidence is the sample is: ', df.target.mean())

Loan delinquencies account for 19.5% instances. This is an imbalanced dataset. We should keep this in mind while performing model evaluation.

In [ ]:
cat_features_te = ['sub_grade', 'emp_title', 'purpose', 'title', 'zip_code', 'addr_state', 'grade', 'home_ownership']
cat_features_ohe = ['verification_status', 'initial_list_status', 'application_type']

The code above lists categorical columns, for which I will use target encoding or one-hot encoding.

### 4. EDA

In this subsection I look at the distribution of key loan features.

In [ ]:
display(df[['loan_amnt', 'funded_amnt', 'funded_amnt_inv']].describe())
display(df.term.value_counts())

So the three amount features above are basically identical.

In [ ]:
sns.countplot(y='purpose', data=df)
plt.show()
sns.countplot(y='emp_length', data=df)
plt.show()
sns.histplot(x='lti', data=df).set(title='Loan LTI')
plt.show()
sns.histplot(x='iti', data=df).set(title='Loan ITI')
plt.show()
sns.histplot(x='annual_inc', data=df, log_scale=True).set(title='Annual Income')
plt.show()
sns.histplot(x='dti', data=df).set(title='Loan DTI')
plt.show()
sns.histplot(x='grade', data=df).set(title='Loan Grade')
plt.show()

Debt consolidation and credit card refinancing are the most frequent purposes for loans. It makes sense given maximum \\$40,000 loan size. Annual income is very approximately Gaussian and is centered around \\$100,000.
Key credit metrics as a fraction of income have pretty tight distributions with few outliers. It makes sense, since people with little to no income are unlikely to be good borrowers.

### EDA with target

Here I explore univariate relations between target and key features.

In [ ]:
df.open_acc = df.open_acc.astype(int)
df.delinq_2yrs = df.delinq_2yrs.astype(int)
df.pub_rec = df.pub_rec.astype(int)
df.fico_range_high = df.fico_range_high.astype(int)

sns.set(rc={'figure.figsize':(12,8)})
sns.barplot(x='grade', y='target', data=df)
plt.show()
sns.barplot(x='sub_grade', y='target', data=df)
plt.show()
sns.barplot(x='home_ownership', y='target', data=df)
plt.show()
sns.barplot(x='delinq_2yrs', y='target', data=df)
plt.show()
sns.barplot(x='pub_rec', y='target', data=df)
plt.show()
sns.barplot(x='pub_rec_bankruptcies', y='target', data=df)
plt.show()
sns.set(rc={'figure.figsize':(20,8)})
sns.barplot(x='open_acc', y='target', data=df)
plt.show()
#sns.barplot(x='total_acc', y='target', data=df)
#plt.show()
sns.barplot(x='fico_range_high', y='target', data=df)
plt.show()
sns.barplot(x='purpose', y='target', data=df)
plt.show()
sns.barplot(x='year_issued', y='target', data=df)
plt.show()

Unsurprisingly, default rate is strongly positively correlated with loan grade. Low-grade loans are much more likely to default. Similarly, higher FICO ratings lead to lower default probability

In [ ]:
a = df.dti
b = df.target
bins = np.linspace(a.min(), a.max() + 1e-12, 51) 
c = np.digitize(a, bins)
plt.bar(bins[:-1], [np.mean(b[c == i]) for i in range(1, len(bins))],
        width=bins[1] - bins[0], align='edge', fc='turquoise', ec='black')
plt.xticks(bins)
plt.margins(x=0.02) # smaller margins
plt.show()

a = df.int_rate
b = df.target
bins = np.linspace(a.min(), a.max() + 1e-12, 26) 
c = np.digitize(a, bins)
plt.bar(bins[:-1], [np.mean(b[c == i]) for i in range(1, len(bins))],
        width=bins[1] - bins[0], align='edge', fc='turquoise', ec='black')
plt.xticks(bins)
plt.margins(x=0.02) # smaller margins
plt.show()


There is strong positive relation between interest rate and default probability. This is to be expected, since otherwise it would mean very bad internal risk modeling at Lending Club.

There are interesting results regarding debt-to-income ratio. Default probability is predictably rising until DTI reaches 40%. Then it levels off and even to seem to fall after 60%. Due to sall number of loans with such a high DTI, it is hard to say whether this pattern is statistically significant.

### 5. Train-test split, missing values and feature engineering

In this section I fill missing values from train set and encode categorical features.

For categorical features, which will be target-encoded later, I fill missing values with 'MISSING' category.

To mitigate effect of outliers, I use median to impute values of missing numerical features.

In [ ]:
#display(df.count())

features_fill_M = ['emp_title', 'title']
features_fill_med = ['dti', 'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_util', 
                     'total_acc', 'mort_acc', 'pub_rec_bankruptcies', 'bc_open_to_buy','tot_cur_bal_inc', 'total_rev_inc',
                    'total_bal_ex_mort_inc', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'avg_cur_bal_inc',
                    'mo_sin_rcnt_rev_tl_op', 'num_actv_rev_tl', 'num_op_rev_tl', 'tax_liens']
features_fill_zero = ['open_rv_12m', 'open_il_12m', 'emp_length', 'num_tl_90g_dpd_24m', 
                      'num_tl_30dpd', 'num_tl_120dpd_2m', 'num_accts_ever_120_pd',
                     'acc_open_past_24mths', 'tot_coll_amt']

for col in features_fill_zero:
    df[col] = df[col].fillna(value=0)

for col in features_fill_M:
    df[col] = df[col].cat.add_categories(['MISSING']) 
    df[col] = df[col].fillna(value='MISSING')
    
test_size = 0.25
df.reset_index(inplace=True, drop=True)
random.seed(2)
test_index = random.sample(list(df.index), int(test_size*df.shape[0]))
train = df.iloc[list(set(df.index)-set(test_index))]
test = df.iloc[test_index]
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
test00 = test.copy()
train.drop(columns=['id'],inplace=True)
test.drop(columns=['id'],inplace=True)
display(train.shape, test.shape, train.head(3), test.head(3))

for col in features_fill_med:
    train[col] = train[col].fillna(train[col].median())
    test[col] = test[col].fillna(train[col].median())
    
train['total_rev_hi_lim'] = train.total_rev_inc.median()*train.annual_inc
test['total_rev_hi_lim'] = train.total_rev_inc.median()*test.annual_inc

train['total_bal_ex_mort'] = train.total_bal_ex_mort_inc.median()*train.annual_inc
test['total_bal_ex_mort'] = train.total_bal_ex_mort_inc.median()*test.annual_inc

train['tot_cur_bal'] = train.tot_cur_bal_inc.median()*train.annual_inc
test['tot_cur_bal'] = train.tot_cur_bal_inc.median()*test.annual_inc

train['avg_cur_bal'] = train.avg_cur_bal_inc.median()*train.annual_inc
test['avg_cur_bal'] = train.avg_cur_bal_inc.median()*test.annual_inc

display(train.count())

### Categorical features encoding

Depending of feature type, I use three encoding types:

- Target encoding. I do it for feature with more than 5 unique values.
- One-hot encoding. I use it for features with less than 5 unique values.
- Frequency encoding. 'Title' feature only.

In [ ]:
# TE for categorical features

time1 = time.time()
encoder = CrossFoldEncoder(MEstimateEncoder, m=10)
train_encoded = encoder.fit_transform(train, train.target, cols=cat_features_te)
test_encoded = encoder.transform(test)

freq_enc = (train.groupby('title').size()) / len(train)
train['title_fencoded'] = train['title'].apply(lambda x : freq_enc[x])
test['title_fencoded'] = test['title'].apply(lambda x : freq_enc[x])

train.drop(columns=cat_features_te, inplace=True)
test.drop(columns=cat_features_te,  inplace=True)
train = pd.concat([train, train_encoded], axis = 1)
test = pd.concat([test, test_encoded], axis = 1)

display(time.time()-time0, time.time()-time1)
display(train.shape, train.head(), train.count())
train0 = train.copy()
test0 = test.copy()

In [ ]:
X_train = train.copy()
y_train = X_train.pop('target')
X_test = test.copy()
y_test = X_test.pop('target')
display(X_test.head())

### Do OHE for some features ###

feature_transformer = ColumnTransformer([
    ("cat", OneHotEncoder(sparse = False, handle_unknown="ignore", drop='if_binary'), cat_features_ohe)], remainder="passthrough")

print('Number of features before transaformation: ', X_train.shape)
X_train = pd.DataFrame(feature_transformer.fit_transform(X_train), columns=feature_transformer.get_feature_names_out())
X_test = pd.DataFrame(feature_transformer.transform(X_test), columns=feature_transformer.get_feature_names_out())
X_train.columns = X_train.columns.str.replace(r'^cat__', '').str.replace(r'^remainder__', '')
X_test.columns = X_test.columns.str.replace(r'^cat__', '').str.replace(r'^remainder__', '')

print('time to do feature proprocessing: ', time.time()-time1)
print('Number of features after transaformation: ', X_train.shape)
X_train.head()
X_test_0 = X_test.copy()

### 6. Modeling

Here I will build 2 models to predict loan bankruptcies:

- Baseline XGBoost model (default hyperparameters).
- XGBoost model with hyperparameter optimization using Optuna.

In [ ]:
time1 = time.time()
xgb = XGBClassifier(tree_method = 'gpu_hist')
xgb.fit(X_train, y_train)

precision_t, recall_t, threshold = precision_recall_curve(y_train, xgb.predict_proba(X_train)[:, 1])
auc_precision_recall_train = auc(recall_t, precision_t)
temp = recall_t[(recall_t>0.095)&(recall_t<0.105)]
temp = temp[int(len(temp)/2)]
indexx = ((np.where(recall_t==temp)))[0][0]
r10prec_train = precision_t[indexx]
precision_t, recall_t, threshold = precision_recall_curve(y_test, xgb.predict_proba(X_test)[:, 1])
auc_precision_recall_test = auc(recall_t, precision_t)
temp = recall_t[(recall_t>0.095)&(recall_t<0.105)]
temp = temp[int(len(temp)/2)]
indexx = ((np.where(recall_t==temp)))[0][0]
r10prec_test = precision_t[indexx]

display('Train Accuracy: ', accuracy_score(y_train,xgb.predict(X_train)))
display('F1 score: ', f1_score(y_train,xgb.predict(X_train)))
display('ROCAUC: ', roc_auc_score(y_train,xgb.predict(X_train)))
display('PRAUC: ', auc_precision_recall_train)
display('R10P: ', r10prec_train)

# Performance evaluation:
display('Test Accuracy: ', accuracy_score(y_test,xgb.predict(X_test)))
display('F1 score: ', f1_score(y_test,xgb.predict(X_test)))
display('ROCAUC: ', roc_auc_score(y_test,xgb.predict(X_test)))
display('PRAUC: ', auc_precision_recall_test)
display('R10P: ', r10prec_test)
display(time.time()-time1)

fig, ax = plt.subplots()
ax.plot(recall_t, precision_t, color='purple')
ax.set_title('Precision-Recall Curve, test')
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')
ax.set_ylim(bottom=0, top=1.02)
plt.show()

In order to save computing time for users of this notebook, I save hyperparameter values, which Optuna converged upon. If you want to run hyperparameter optimization yourself, uncomment the code, performing hyperparameter optimization below. That is, everything between 'def objective' and 'optuna_hyperpars' two cells below. Due to random elements of XGBoost, the results may be somwhat different.

In [ ]:
# here are starting hyperparameters for Optuna optimization

starting_hyperparameters = {'n_estimators': 1186,
 'max_depth': 4,
 'learning_rate': 0.04582452146963614,
 'colsample_bytree': 0.600410879432218,
 'subsample': 0.5561312005431341,
 'alpha': 5.0981168922187665,
 'lambda': 4.090612170344411,
 'gamma': 0.0029390495081042726,
 'min_child_weight': 0.21653462492301606,
 'scale_pos_weight': 1,
 'tree_method': 'gpu_hist'}

In [ ]:
### Fit XGBoost using Optuna hyperparameter optimization ###

time1 = time.time()

# def objective(trial, cv_runs=1, n_splits=2, n_jobs=-1):

#     cv_regularizer=0.01
#     # Usually values between 0.1 and 0.2 work fine.

#     params = {
#         "tree_method": 'gpu_hist',
#         "verbosity": 0,  # 0 (silent) - 3 (debug)
#         "n_estimators": trial.suggest_int("n_estimators", 500, 1500),
#         "max_depth": trial.suggest_int("max_depth", 2, 6),
#         "learning_rate": trial.suggest_uniform("learning_rate", 0.005, 0.2),
#         "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.1, 0.95),
#         "subsample": trial.suggest_uniform("subsample", 0.5, 0.95),
#         "alpha": trial.suggest_loguniform("alpha", 0.1, 10.0),
#         "lambda": trial.suggest_loguniform("lambda", 0.1, 150.0),
#         "gamma": trial.suggest_loguniform("gamma", 1e-10, 10.0),
#         "min_child_weight": trial.suggest_loguniform("min_child_weight", 0.1, 10),
#         "n_jobs": n_jobs,
#         "scale_pos_weight": trial.suggest_uniform("scale_pos_weight", 1, 3),
#     }
#     # usually it makes sense to resrtict hyperparameter space from some solutions which Optuna will find
#     # e.g., for tmx-joined data only (downsampled tmx), optuna keeps selecting depths of 2 and 3.
#     # for my purposes (smooth left side of prc, close to 1), those solutions are no good.

#     temp_out = []

#     for i in range(cv_runs):

#         X = X_train
#         y = y_train

#         model = XGBClassifier(**params)
#         rkf = KFold(n_splits=n_splits, shuffle=True)
#         X_values = X.values
#         y_values = y.values
#         y_pred = np.zeros_like(y_values)
#         y_pred_train = np.zeros_like(y_values)
#         for train_index, test_index in rkf.split(X_values):
#             X_A, X_B = X_values[train_index, :], X_values[test_index, :]
#             y_A, y_B = y_values[train_index], y_values[test_index]
#             model.fit(X_A, y_A, eval_set=[(X_B, y_B)], verbose = False)
#             y_pred[test_index] += model.predict_proba(X_B)[:, 1]
#             #y_pred_train[train_index] += model.predict_proba(X_A)[:, 1]
                      
#         #precision_t, recall_t, threshold = precision_recall_curve(y_train, y_pred_train)
#         #score_train = auc(recall_t, precision_t)
#         precision_t, recall_t, threshold = precision_recall_curve(y_train, y_pred)
#         score_test = auc(recall_t, precision_t)
            
#         #score_train = roc_auc_score(y_train, y_pred_train)
#         #score_test = roc_auc_score(y_train, y_pred) 
#         #overfit = score_train-score_test
#         #temp_out.append(score_test-cv_regularizer*overfit)
#         temp_out.append(score_test)

#     return (np.mean(temp_out))

# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=50)

# # study = optuna.create_study(direction="maximize")
# # study.enqueue_trial(starting_hyperparameters)
# # study.optimize(objective, n_trials=50)

# print('Total time for hypermarameter optimization ', time.time()-time1)
# hp = study.best_params
# for key, value in hp.items():
#     print(f"{key:>20s} : {value}")
# print(f"{'best objective value':>20s} : {study.best_value}")

# optuna_hyperpars = study.best_params
# optuna_hyperpars['tree_method']='gpu_hist'
# optuna_hyperpars['scale_pos_weight']=1

optuna_hyperpars = starting_hyperparameters

optuna_xgb = XGBClassifier(**optuna_hyperpars, seed=8)
optuna_xgb.fit(X_train, y_train)

precision_t, recall_t, threshold = precision_recall_curve(y_train, optuna_xgb.predict_proba(X_train)[:, 1])
auc_precision_recall_train = auc(recall_t, precision_t)
temp = recall_t[(recall_t>0.095)&(recall_t<0.105)]
temp = temp[int(len(temp)/2)]
indexx = ((np.where(recall_t==temp)))[0][0]
r10prec_train = precision_t[indexx]

fig, ax = plt.subplots()
ax.plot(recall_t, precision_t, color='purple')
ax.set_title('Precision-Recall Curve, train')
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')
ax.set_ylim(bottom=0, top=1.02)
plt.show()

precision_t, recall_t, threshold = precision_recall_curve(y_test, optuna_xgb.predict_proba(X_test)[:, 1])
auc_precision_recall_test = auc(recall_t, precision_t)
temp = recall_t[(recall_t>0.095)&(recall_t<0.105)]
temp = temp[int(len(temp)/2)]
indexx = ((np.where(recall_t==temp)))[0][0]
r10prec_test = precision_t[indexx]

fig, ax = plt.subplots()
ax.plot(recall_t, precision_t, color='purple')
ax.set_title('Precision-Recall Curve, test')
ax.set_ylabel('Precision')
ax.set_xlabel('Recall')
ax.set_ylim(bottom=0, top=1.02)
plt.show()

display('Train Accuracy: ', accuracy_score(y_train,optuna_xgb.predict(X_train)))
display('F1 score: ', f1_score(y_train,optuna_xgb.predict(X_train)))
display('ROCAUC: ', roc_auc_score(y_train,optuna_xgb.predict(X_train)))
display('PRAUC: ', auc_precision_recall_train)
display('R10P: ', r10prec_train)
# Performance evaluation:
display('Test Accuracy: ', accuracy_score(y_test,optuna_xgb.predict(X_test)))
display('F1 score: ', f1_score(y_test,optuna_xgb.predict(X_test)))
display('ROCAUC: ', roc_auc_score(y_test,optuna_xgb.predict(X_test)))
display('PRAUC: ', auc_precision_recall_test)
display('R10P: ', r10prec_test)
display('Time to do hyperparameter optimization: ', time.time()-time1)

Optuna hyperparameter optimization delivers significant improvement in a performance over baseline XGBoost model. Due to faster convergence and faster runtime I prefer Optuna to GridSearch or RandomizedSearch.

### 7. Model interpretation

In this section I use SHAP library in order to provide interpretation of the results.

Feature importances tell us which features give the model its predictive power. SHAP values indicate how certain values of each feature affect model predictions.

In [ ]:
# template here: https://www.kaggle.com/code/kaanboke/catboost-lightgbm-xgboost-explained-by-shap/notebook
explainerxgbc = shap.TreeExplainer(optuna_xgb)
shap_values_XGBoost_test = explainerxgbc.shap_values(X_test)
shap_values_XGBoost_train = explainerxgbc.shap_values(X_train)

vals = np.abs(shap_values_XGBoost_test).mean(0)
feature_names = X_test.columns
feature_importance = pd.DataFrame(list(zip(feature_names, vals)),
                                 columns=['col_name','feature_importance_vals'])
feature_importance.sort_values(by=['feature_importance_vals'],
                              ascending=False, inplace=True)
#display(feature_importance)

shap.summary_plot(shap_values_XGBoost_test, X_test, plot_type="bar", plot_size=(8,8), max_display=30)
shap.summary_plot(shap_values_XGBoost_train, X_train,plot_type="dot", plot_size=(8,8), max_display=20)

We can see that subgrade is the most important feature. SHAP values of encoded subgrade are positively related to loan default risk. This is to beexpected, since by construction target encodings measure average value of a target. So we would expect all target-encoded features to positively predict defaults. 

The other important features are loan term, number of accounts, opened in the past 24 months, LTI (Loan-to_Income ratio) and DTI (Debt-toIncome ratio). Their partial efects make sense: Higher LTI, DTI and higher number of recently opened accounts are positively related to loan defaults.

The code below explains model predictions for a specific instances. For example, it illustrates why the model predicts no default for a loan at index 1 and default for a loan with index 10.

In [ ]:
indx = 1
#fig = plt.subplots(figsize=(2,2),dpi=200)
ax_2= shap.decision_plot(explainerxgbc.expected_value, shap_values_XGBoost_test[indx], X_test.iloc[indx],link= "logit")
shap.initjs()
shap.force_plot(explainerxgbc.expected_value, shap_values_XGBoost_test[indx], X_test.iloc[[indx]],link= "logit")

This loan has high subgrade and low interest rate. Its other features are the features, common for well-performing loans. Thus model confidently predicts no default.

In [ ]:
indx = 10
#fig = plt.subplots(figsize=(2,2),dpi=200)
ax_2= shap.decision_plot(explainerxgbc.expected_value, shap_values_XGBoost_test[indx], X_test.iloc[indx],link= "logit")
shap.initjs()
shap.force_plot(explainerxgbc.expected_value, shap_values_XGBoost_test[indx], X_test.iloc[[indx]],link= "logit")

This loan looks very different. It has very low rating (i.e., subgrade), high LTI, high interest rate and long maturity. These features explain why the model assigns default probability of 55% to this loan.

### 8. Business implications

#### 8.1 Is this model useful?

This project is a perfect illustration of how machine learning can create value if we think hard about business problem at hand rather than focusing on a few narrow model evaluation metrics.

If you view this project as a purely data mining/prediction exercise, then it is arguably a failure. Model accuracy is 81% which barely beats 80% from a trivial model, always predicting no default. F1 score is only 18%, which is not impressive. ROCAUC is 55% which is barely better than a random guess.

However, the predictive model above can create value if used properly. Let me show how.

Precision-recall curve above shows that model is actually very good in predicting defaults for around 20% of defaulted loans. For example, its precision at 10% recall is 60%. This means that if we use the model only to predict default for those loans it is most confident about, we can avoid at least 10% of credit losses while forgoing relatively few profitable lending opportunities. 

#### 8.2 How much value does it create?

To quantify value created by the model, I consider a problem faced by all Lending Club investors as a group to maximize their investment profit. They can do so by avoiding x% of the riskiest loans as determined by my model. In order to keep this analysis simple and accessible to wider audience I abstract away from time discounting, compound interest and present value calculations. All the calculations below assume simple interest.

The decision of an investor to avoid financing x% of the riskiest loans is a tradeoff between a foregone interest income and a credit loss. 

If investors invest in a loan:
- They receive interest income in the future as long as the borrower repays a loan.
- They suffer credit losses and receive only recovered amount if the borrower defaults.

So their total dollar return (TR) from investment is as follows:

$$ TR_{Invest} = \sum_{j \in RepaidLoans}LoanAmount_j*(r_j+1) + \sum_{k \in DefaultedLoans}Recovery_k.$$

Their outisde riskless option is to invest in Treasury bills with similar maturity. Average yield of 3-year treasury notes over this sample period [is around 2%](https://fred.stlouisfed.org/series/DGS3). So their TR if they forgo such loans is:

$$ TR_{Forgo} = \sum_{i \in AllLoans} LoanAmount_i*(1.02).$$

Thus the decision to issue such loans depends on which part dominates. Savings of investors from avoiding top x% riskiest loans are the difference between the returns fom investing in T-notes and LendingClub loans:

$$ \sum_{i \in AllLoans} LoanAmount_i*(1.02) - \sum_{j \in RepaidLoans}LoanAmount_j*(r_j+1) + \sum_{k \in DefaultedLoans}Recovery_k $$

The expression above is the value created from avoiding the riskiest loans. For example, if I use the model to identify top 10% riskiest loans, then AllLoans will be the loans with 10% of the highest predicted default probabilities. RepaidLoans and DefaultedLoans will be subsets of those loans.

The code below uses loan_amnt, int_rate and recovery features to calculate this value created. Risky loans are defined based on threshold, necessary to reach 10% recall.

In [ ]:
recoveries['total_recovery'] = recoveries.total_rec_prncp + recoveries.total_rec_int + recoveries.recoveries
recoveries['tot_recov_rp'] = recoveries.total_recovery/recoveries.loan_amnt
recoveries['tot_recov_rt'] = recoveries.total_recovery/(recoveries.loan_amnt*((recoveries.int_rate/100+1)**3))
# in a few cases the recoveries seem to exceed total proceeds.
# this may be the result of ignoring time discounting and compound interest in my calculations
# ior those may be rarer 60-months loans. so i do not adjust such cases.
display(recoveries.describe(), recoveries.head())

In [ ]:
X_test = X_test_0.copy()
test = X_test[['loan_amnt', 'int_rate']]
test.int_rate = test.int_rate/100+1
test['y_pred'] = optuna_xgb.predict_proba(X_test)[:,1]
test['id'] = test00.id
test['y'] = y_test
test = pd.merge(test, recoveries[['id', 'loan_amnt', 'int_rate', 'total_recovery']], on='id', how = 'left')
display(recoveries.head(), test.head())
display(test.loc[test.y==0].count(), test.loc[test.y==1].count())

In [ ]:
# select riskiest loans by calculating decision threshold, giving 10% recall:

desired_recall = 0.1

temp = recall_t[(recall_t>(desired_recall-0.001))&(recall_t<(desired_recall+0.001))]
temp = temp[int(len(temp)/2)]
indexx = ((np.where(recall_t==temp)))[0][0]
r10threshold = threshold[indexx]
p90risk = r10threshold
test['p90risk'] = (test.y_pred>=p90risk).astype(int)
display(test.shape, p90risk)

risky_loans = test[test.p90risk==1]
risky_loans.loc[risky_loans.total_recovery.isnull(),'total_recovery']=\
risky_loans.loan_amnt_x*(risky_loans.int_rate_x**3)
# when the loan is repaid, I calculate total return and save it into total_recovery column.
display(risky_loans.head(), risky_loans.shape)

proceeds = risky_loans.total_recovery.sum()
proceeds_tnotes = (risky_loans.loan_amnt_x.sum())*(1.02**3)
print("Investors' total returns from investing in risky loans", int(proceeds))
print("Investors' total returns from investing in T-notes", int(proceeds_tnotes))
print("Investors' savings: $", int(proceeds_tnotes-proceeds))
estimated_savings = (proceeds_tnotes-proceeds)*3.3*4
print("Estimated investors' savings from all LendingClub loans: $", estimated_savings)


# In this particular sample, the savings are maximized at 8.5% recall threshold.
# Then the saving for investors will be $4.9M in test set or 64.6M for all loans.

print(time.time() - time0)

We can see that using this model allows investors in the test sample to save \\$ 4.53M. 
Accounting for 75/25 train-test split and initial 30\% sample from the original dataset, the estimated saving should be $$4.53M*3.3*4 = 59.7M.$$

**The takeaway is that we do not necessarily need ML model with superior predictive performance over the whole feature space. As long as ML model performs well on a subset of instances, it can create great value if used properly**.
